In [74]:
import requests
import pandas as pd
import io
import os
import numpy as np
from tabulate import tabulate
from dotenv import load_dotenv
from IPython.display import HTML
from yahoo_fin.stock_info import get_data

cwd = os.getcwd()
path = os.path.dirname(cwd)
folder_path = os.path.join(path, 'stock_data')


In [75]:
def to_closest_friday(date):
    day_of_week = date.dayofweek
    if day_of_week == 4: 
        return date
    elif day_of_week < 4: 
        return date + pd.Timedelta(days=(4 - day_of_week))
    else: 
        return date - pd.Timedelta(days=(day_of_week - 4))


def get_ticker_list():
    folder_path = os.path.join(path, 'preprocessing_data')
    txt_files = [file for file in os.listdir(folder_path) if file.endswith('.txt')]
    file_contents = {}
    for file in txt_files:
        with open(os.path.join(folder_path, file), 'r', encoding='utf-8') as f:
            file_contents[file] = f.read()
    final_df = []
    for filename, content in file_contents.items():
        final_df.append(pd.read_html(content)[0])
    return final_df

def calculate_wma(data):
    weights = np.arange(1, 31)[::-1]
    return np.dot(data, weights) / weights.sum()

def get_stock_df(ticker):
    df = get_data(ticker,index_as_date = True, interval="1wk")
    df.index = df.index.map(to_closest_friday)
    return df

def calculate_future_average(row, df):
    three_months_later = row.name + pd.DateOffset(weeks=12)
    six_months_later = row.name + pd.DateOffset(weeks=24)
    future_df = df[(df.index > three_months_later) & (df.index <= six_months_later)]
    if df.index[-1] > six_months_later and not future_df.empty:
        return future_df['close'].mean()
    else:
        return None
    
def label_stocks(row):
    if row['three_to_six'] >= row['close'] * 1.1:
        return 2
    elif row['three_to_six'] <= row['close'] * 0.9:
        return 4
    else:
        return 1


In [76]:
dfs = get_ticker_list()
tickers = []
for df in dfs:
    tickers += list(df['Symbol'])
len(tickers)
# print(tickers)
ticker_list = [item for item in tickers if not isinstance(item, float)]
ticker_list.sort()

/tmp/ipykernel_9255/3479759253.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  final_df.append(pd.read_html(content)[0])
/tmp/ipykernel_9255/3479759253.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  final_df.append(pd.read_html(content)[0])
/tmp/ipykernel_9255/3479759253.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  final_df.append(pd.read_html(content)[0])
/tmp/ipykernel_9255/3479759253.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  final_df.append(pd.read_html(content)[

In [77]:
####　EXAMPLE 

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for stock in ticker_list:
    try:
        df = get_stock_df(stock)
        df = df.drop(["adjclose","ticker"], axis = 1)
        df['wma30'] = df['close'].rolling(window=30).apply(calculate_wma, raw=True)
        df['fyh'] = df['high'].rolling(window=5*52, min_periods=1).max()
        df = df.dropna()
        df['three_to_six'] = df.apply(calculate_future_average, df=df, axis=1)
        df = df.dropna()
        df['stage'] = df.apply(label_stocks, axis=1)
        for i in range(1, len(df)):
            if (df.iloc[i-1]['stage'] == 2 or df.iloc[i-1]['stage'] == 3) and df.iloc[i]['stage'] != 4:
                df.iloc[i,8] = 3
        df.to_pickle(folder_path + '/' + stock + '.pkl')
    except:
        print(stock + " not found")

AGM.Anot found
AKO.Anot found
AKO.Bnot found
BF.Anot found
BF.Bnot found
BH.Anot found
BIO.Bnot found
BNRE.Anot found
BRK.Anot found
BRK.Bnot found
CRD.Anot found
CRD.Bnot found
CWEN.Anot found
DYCQnot found
GEF.Bnot found
GTN.Anot found
HEI.Anot found
HVT.Anot found
JVSAnot found
LEGTnot found
LEN.Bnot found
LGF.Anot found
LGF.Bnot found
MKC.Vnot found
MOG.Anot found
MOG.Bnot found
PBR.Anot found
TAP.Anot found
UHAL.Bnot found
WSO.Bnot found


In [ ]:
df = pd.read_pickle(folder_path + '/A.pkl')

In [ ]:
display(df)